## 4-3. 사이킷런으로 카운트 벡터 생성

카운트 기반으로 문서표현을 해서 머신러닝으로 분석을 하고 싶다면, 사이킷런의 텍스트 관련 라이브러리를 쓰면 훨씬 편하다. 사이킷런은 머신러닝을 구현할 수 있는 가장 간편하고 효과적인 파이썬 라이브러리로, 텍스트나 이미지로부터 특성을 추출하는 도구로 sklearn.feature_extraction 모듈이 있으며 사용법과 예제를 [홈페이지](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)에서 상세히 설명한다. 가장 기본적인 도구는 CountVectorizer 클래스이며, 객체를 생성할 때 기본값을 사용해도 되지만 좀 더 섬세하게 조정하고 싶다면 tokenizer, stop_words, ngram_range, max_df, min_df, max_features, vocabulary, binary와 같은 매개변수들을 이용하면 된다. CountVectorizer 클래스의 주요 메서드로는 fit(raw_documents), transform(raw_documents), fit_transform(raw_documents), get_feature_names_out()이 있다.

사이킷런은 자체적인 토크나이저를 지원하므로, 사용자가 별도로 미리 토큰화를 하지 않아도 된다. 그러나 좀 더 세부적인 조정을 통해 성능을 높이고 싶을 때는 토크나이저를 함수로 정의하고 사이킷런에서 이를 사용하게 할 수 있다. 한글의 경우에는 KoNLPy로 형태소 분석을 수행해야 하므로 반드시 별도의 토크나이저를 사용해야 한다.

CountVectorizer의 fit() 메서드는 tokenizer, stop_words, max_df, min_df, max_features 등을 이용해 문서 집합 전체에 대해 토큰화와 불용어 제거, 특성 선택을 수행해 특성 집합을 생성하며, 특성 집합의 각 특성은 단어와 인덱스로 구성된다. 인덱스는 나중에 각 문서별로 특성 벡트를 추출할 때 사용한다. transform() 메서드는 생성된 특성 집합을 이용해 각 문서로부터 특성 벡터 혹은 카운트 벡터를 생성한다. 이때 특성 집합의 인덱스를 사용해 각 단어에 대해 빈도를 표시한다. 예를 들어 '(0, 0):5'에서 첫째 0은 첫 문서임을 나타내고, 둘째 0은 특성 집합의 인덱스를 나타내며, 5는 이 특성의 빈도를 의미한다.

앞선 실습과 달리 이런 식으로 특성 벡터를 표현하는 이유는 특성 벡터가 매우 희소하기 때문이다. 사이킷런과 같이 특성 벡터를 표현하면 값이 0인 대다수의 특성을 굳이 벡터로 표현하지 않아도 되므로 공간을 좀 더 효율적으로 활용할 수 있다. 

이번 예제에서는 앞에서 사용한 movie_reviews에 대해 사이킷런의 CountVectorizer 클래스를 이용해 카운트 벡터를 생성한다.

In [1]:
from nltk.corpus import movie_reviews

# data 준비, movie_reviews.raw()를 사용해 raw text를 추출
reviews = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

CountVectorizer 객체를 생성할 때, vocabulary 매개변수를 쓰면 앞에서 만든 word_features에 있는 단어들만으로 벡터를 구성할 수 있다. 그러나 아무런 전처리 없이 바로 분석을 시작할 때는, 단지 max_features에 빈도가 높은 단어부터 사용할 단어 수를 지정함으로써 원하는 벡터를 구성할 수 있다. word_features를 직접 생성할 때는 빈도 순으로 정렬했으나, 사이킷런에서는 단어들이 알파벳 순으로 정렬된다는 차이가 있다. 아래 코드는 앞의 결과와 비교하려고 word_features를 사용했으며, print로 객체를 출력하면 사용된 인수를 볼 수 있다.

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]

tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents]

word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)

word_features = sorted_features[:1000]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer() # 1. 모든 매개변수에 기본값을 사용하는 경우
cv = CountVectorizer(vocabulary=word_features) # 2. 생성한 word_features로 특성 집합을 지정하는 경우
# cv = CountVectorizer(max_features=1000) # 3. 특성 집합을 지정하지 않고 최대 특성의 수를 지정하는 경우

print(cv)

CountVectorizer(vocabulary=['film', 'one', 'movie', 'like', 'even', 'time',
                            'good', 'story', 'would', 'much', 'also', 'get',
                            'character', 'two', 'well', 'first', 'characters',
                            'see', 'way', 'make', 'life', 'really', 'films',
                            'plot', 'little', 'people', 'could', 'bad', 'scene',
                            'never', ...])


객체를 생성했으면 아래와 같이 fit_transform()으로 특성 집합을 생성하고 카운트 벡터를 생성한다. get_feature_names_out()을 출력해보면 word_features와 사용된 단어 및 순서가 동일한 것을 볼 수 있다.

In [4]:
reviews_cv = cv.fit_transform(reviews) # reviews를 이용해 count vector를 학습하고 변환
print(cv.get_feature_names_out()[:20]) # count vector에 사용된 feature 이름을 반환
print(word_features[:20]) # 비교를 위해 출력

['film' 'one' 'movie' 'like' 'even' 'time' 'good' 'story' 'would' 'much'
 'also' 'get' 'character' 'two' 'well' 'first' 'characters' 'see' 'way'
 'make']
['film', 'one', 'movie', 'like', 'even', 'time', 'good', 'story', 'would', 'much', 'also', 'get', 'character', 'two', 'well', 'first', 'characters', 'see', 'way', 'make']


이제 reviews_cv(리뷰 문서에 대한 카운트 벡터)가 어떻게 생겼는지 알아본다. 

In [5]:
print('# type of count vectors:', type(reviews_cv))
print('# shape of count vectors:', reviews_cv.shape)

# type of count vectors: <class 'scipy.sparse.csr.csr_matrix'>
# shape of count vectors: (2000, 1000)


위 결과를 보면 이 벡터의 타입은 'scipy.sparse.csr.csr_matrix'라고 나온다. shape은 (2000, 1000)으로 나오는데, 이는 리뷰의 수가 2,000개이고 각 리뷰마다 특성의 수가 1,000개이기 때문이다. 이것만 봐서는 넘파이의 ndarray와 비슷하게 느껴지는데, 실제로 첫째 리뷰의 앞 특성 10개만 출력해본다.

In [6]:
print('# sample of count vectors:')
print(reviews_cv[0, :10])

# sample of count vectors:
  (0, 0)	6
  (0, 1)	3
  (0, 2)	6
  (0, 3)	3
  (0, 4)	3
  (0, 6)	2
  (0, 8)	1


위 결과를 보면 왼쪽의 (0, 0)은 좌표이고 오른쪽의 숫자는 빈도수로 reviews_cv가 ndarray와는 다른 것을 알 수 있다. 자세히 보면 (0, 5) 등이 빠져 있는 것을 볼 수 있는데, 이는 카운트 벡터가 매우 희소해 값이 있는 특성만 저장한 것임을 알 수 있다. 아래 예시를 보면 더 명확히 이해 가능하다.

In [7]:
reviews_cv

<2000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 252984 stored elements in Compressed Sparse Row format>

위와 같이 print를 사용하지 않고 reviews_cv를 바로 입력하면 print(repr(reviews_cv))를 입력한 것과 동일하게 몇 가지 정보를 보여주는데, 이 행렬은 2000\*1000 크기이고 실제 그 안에는 252,984개의 값이 있다는 것을 알 수 있다. 이를 제외한 나머지는 값이 0이며, 비율을 계산해보면 252984/(2000\*1000)=12.65%가 나온다. 이 행결은 희소해서 값을 가지는 특성이 그리 많지 않다는 것을 알 수 있으며, 상위 1,000개 대신 10,000개나 그 이상을 선택했다면 훨씬 더 희소했을 것이다. 사이킷런은 이와 같이 희소한 행렬을 효과적으로 관리하려고 Compressed Sparse Row format이라는 데이터 타입을 사용한다. 이 포맷을 넘파이 형태로 변경하고 싶다면 toarray() 메서드를 쓰면 된다.

In [8]:
def document_features(document, word_features):
    word_count = {}
    for word in document:
        word_count[word] = word_count.get(word, 0) + 1
    
    features = []
    for word in word_features:
        features.append(word_count.get(word, 0))
    
    return features

word_features_ex = ['one', 'two', 'teen', 'couples', 'solo']
doc_ex = ['two', 'two', 'couples']
print(document_features(doc_ex, word_features_ex))

feature_sets = [document_features(d, word_features) for d in tokens]

[0, 2, 0, 1, 0]


In [9]:
print(feature_sets[0][:20]) # 앞에서 직접 계산한 카운트 벡터

print(reviews_cv.toarray()[0, :20]) # 변환된 결과의 첫째 feature set 중에서 앞 20개를 출력

[5, 3, 6, 3, 3, 0, 2, 0, 1, 0, 1, 3, 1, 2, 1, 0, 1, 2, 3, 5]
[6 3 6 3 3 0 2 0 1 0 1 3 2 2 1 0 1 2 3 5]


위 결과와 직접 만든 벡터를 비교하면 다 동일한데, 첫 값과 중간의 한 값이 1씩 차이 나는 것을 볼 수 있다. 이는 직접 정규표현식으로 만든 토크나이저와 CountVectorizer가 사용하는 토크나이저에 차이가 있기 때문인데, 중요한 것은 CountVectorizer의 변환 결과인 reviews_cv가 어떤 모양을 가지고 아떤 값을 표현하는지를 명확하게 이해하는 것이다.

get_feature_names_out()이 반환하는 특성인 단어의 순서들은 reviews_cv에 있는 특성 값들의 순서와 일치한다. 따라서 우리가 했던 것처럼 각 특성 값을 단어와 함께 보고 싶다면 아래와 같이 할 수 있다.

In [10]:
for word, count in zip(cv.get_feature_names_out()[:20], reviews_cv[0].toarray()[0, :20]):
    print(f'{word}:{count}', end=', ')

film:6, one:3, movie:6, like:3, even:3, time:0, good:2, story:0, would:1, much:0, also:1, get:3, character:2, two:2, well:1, first:0, characters:1, see:2, way:3, make:5, 

DTM(Document Term Matrix)은 문서를 행으로, 단어를 열로 해서 단어이 빈도를 나타낸 행렬을 말한다. 즉, 위에서 우리가 CountVectorizer로 생성한 reviews_cv 행렬이 이에 해당한다. reviews_cv의 shape를 보면 (2000, 1000)으로 행은 리뷰 수, 열은 우리가 결정한 특성의 수다. 반면 TDM(Term Document Matrix)은 DTM의 전치행렬을 말한다. 즉, 행과 열을 바꾼 것으로 대각선을 축으로 반사시킨 결과라고 할 수 있다.